# OLAP Schemas

### Introduction

In previous lessons, we reviewed online transactional processing (OLTP) databases, and saw that they are strructured for fast transactions -- that is a large number of reads and writes to the database.  But, when performing analytics, we have a different set of priorities.  This warrants a different way of structuring our tables, called the star schema.

### Problems with OLTP

Before getting to the star schema, let's talk through some of the reasons for making a change.  To start, let's think of how the context changes when we move to analytical context.

* When working with an analytical database, we perform fewer transactions.  That is, we won't have to worry about a large number of inserts or updates to the database, as our users will not directly interact with the analytical database.

* Instead of optimizing for inserts and updates, we can optimize for *queries* needed to explore and answer questions about our data.  Queries like these become more time intensive if they require joining together many tables.  And this would occur often if we stuck with an OLTP structure, which has small, discrete tables.  

* The users of an analytical database include business people, who may find the structure of a normalized database, with many small tables confusing.  For example, it is easier to query a user's location and automatically get their city, state and zip from the same table.  With an OLTP structure in 3NF, the city, state and zip each be in their own dedicated table that needs to be joined together to see this information.

So in an analytical context, we no longer need to optimize for inserts and updates, where smaller tables with few columns perform well.  Instead, our use case is in *querying* our data, where we prefer to have larger tables to avoid costly joins.  In addition, a simpler table structure (with a fewer number of tables) would make for a more understandable schema, which is good for business users performing queries on the database.

### The Star Schema - What is it?

Let's see an example of what it looks like when we shift our database structure from a normalized OLTP database to an OLAP database.  For example, this was the OLTP structure of our pagila database.

> <img src="./pagila_schema.png" width="90%">

So with the OLTP structure, we did a good job of normalizing the data -- separating out zipcode, cities, and states to avoid duplicating the same data.  We also have a many to many relationship -- a film can be in multiple stores through the inventory table.  So with the inventory table, we see all of the locations of a given film, or all of the films in a given location.

Now let's look at the star schema.

> <img src="./star_schemad_movies.png" width="60%">

Here we place sales (previously rentals) directly in the center of the schema.  The surrounding tables answer questions of who, what, where, and when.  We can think of the star schema as consisting of two kinds of tables - a fact table at the center, and dimension table branching out of the fact table.

* The fact table 

The fact table is generally an **event** that we are trying to measure and often optimize.  For example, here it's revenue, and on the sales table `price` provides that information.  Now the *context* of that target comes from the surrounding tables, providing information about who, what, where, and when this sale was involved.  

> So, on our fact table, *every* column is either a foreign key or a metric we are tracking.

* The dimension tables 

Surrounding the fact table are the dimension tables.  The dimension tables provide *context* to the event. Notice that the dimension table consists of more descriptive attributes, often of type text.  For example, our dimension table of store, has text fields of manager first and last name, and city, state, and zipcode.

* Star Schema: But Why?

Before moving on, let's remember why were doing this.  With the star schema, any information we want is just one table away.  This makes our schema both more understandable, and our queries less costly to perform.  We now have data repeated througout records, but remember that with this database we won't be updating records -- so there's less of a cost to have duplicative information.  

### Comparing the OLTP and OLAP schemas

Let's take another look at the star schema, considering some of the rules we needed to relax to get to this structure.

> <img src="./star_schemad_movies.png" width="60%">

So the structure above is obviously simpler -- but with the star schema, we now have a structure that will repeat information.  

* The entirety of the `dates` table, to begin with, is not normalized.  Every column is a derivative of the dates column and violates *single source of truth*
* We collapsed the film-store relationship so that we no longer track inventory.  This is because *sales* is the center of our queries.  We care about the movies involved in the sale, not queries about inventory.  
* We moved the manager information about a store to the same table as the store itself.
* And we no longer give city, state, and zip their own table.  This will lead to a duplication of values across our tables.

So in summary, we will not have a *single source of truth* with our OLAP schema.  But by relaxing this restriction, we have fewer tables, and each table is just one step removed from the fact table of sales.  This should make our queries both simpler to write, and less expensive.

### Summary

In this lesson, we saw the differences we get when moving from an OLTP to an OLAP structure.  With the OLAP structure, we use the star schema.  Here we have a fact table at the center, usually describing an event.  Every column of the fact table is either a numeric value or a foreign key to a dimension table.  Our dimension tables provide context to the event, describing information about who, what, where, and when the event occurred.  
We can see that the star schema makes our information simpler and easier to query.  This is preferable when we are no longer making many updates to our data, so do not need to optimize for the speed of transactions or maintaining a single source of truth.  Instead, we prefer to make our queries easier to perform, and faster to perform.  And because the star schema has fewer tables that we need to join together, we generally achieve this with this structure.

### Resources

* [Snowflake Schema](https://en.wikipedia.org/wiki/Snowflake_schema)
* [Star Schema](https://en.wikipedia.org/wiki/Star_schema)